# Training Notebook for a Binary Classification using the Instant Time Approach

### Import Dependencies

In [4]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
import os
from utils import *
import json
import random

model_output_path = 'models'
os.makedirs(model_output_path, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
pl.seed_everything(42)
torch.backends.cudnn.deterministic = True

Seed set to 42


### Datasets (k = 12 Folds Cross Validation) & Training of Encoder + MLP

In [6]:
folds = 12
exp_names_per_fold = 3
labels_to_include = ['ETHANOL', 'ACETONE']
hidden_dim = 64
MAX_EPOCHS = 25

df = pd.read_csv("original_training_set_filtered.csv")
df = df[df['label'].isin(labels_to_include)]

exp_labels = df.groupby("exp_name")["label"].first()
label_encoder = LabelEncoder()
label_encoder.fit(exp_labels.values)
exp_name_to_label = exp_labels.to_dict()

In [ ]:
splits_path = r"C:\Users\miche\OneDrive\Desktop\tesi\repo-tesi\time_windows_approach\models\binaryETHvsACE\fold_splits_exp_names.json"
splits = json.load(open(splits_path, "r"))
splits = {int(k): v for k, v in splits.items()}

for fold in range(1, folds+1):

    # Fold Splitting
    df_train = df[df["exp_name"].isin(splits[fold]["train"])]
    df_val = df[df["exp_name"].isin(splits[fold]["val"])]
    df_test = df[df["exp_name"].isin(splits[fold]["test"])]

    feature_columns = [col for col in df_train.columns if col not in ["label", "exp_name"]]
    scaler = StandardScaler()
    scaler.fit(df_train[feature_columns])
    mean = scaler.mean_
    std = scaler.scale_

    train_dataset = SensorDataset(df = df_train,
                                  label_encoder=label_encoder,
                                  exp_name_to_label=exp_name_to_label,
                                  mean=mean,
                                  std=std)
    
    val_dataset = SensorDataset(df = df_val,
                                label_encoder=label_encoder,
                                exp_name_to_label=exp_name_to_label,
                                mean=mean,
                                std=std)
    
    test_dataset = SensorDataset(df = df_test,
                                 label_encoder=label_encoder,
                                 exp_name_to_label=exp_name_to_label,
                                 mean=mean,
                                 std=std)
    
    print(f"\nFold {fold}/{folds} -> Train: {len(train_dataset)} (exp: {len(df_train['exp_name'].unique())}), ",
        f"Val: {len(val_dataset)} (exp: {len(df_val['exp_name'].unique())}), ",
        f"Test: {len(test_dataset)} (exp: {len(df_test['exp_name'].unique())})")
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, persistent_workers=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)

    model = MLPClassifier(
        num_features=len(feature_columns),
        hidden_dim=hidden_dim,
        num_classes=2,
        lr=1e-4,
        dropout=0.2,
        weight_decay=1e-5
    )

    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        patience=5,
        verbose=False,
        mode="min"
    )

    trainer = Trainer(
        max_epochs=MAX_EPOCHS,
        callbacks=[early_stop_callback],
        enable_model_summary=False,  # <- Hides the model summary
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1
    )

    trainer.fit(model, train_loader, val_loader)
    # Save the model
    output_name = f"binaryETHvsACE/binaryETHvsACE_MLP_h{hidden_dim}_{fold}.ckpt"
    model_path = os.path.join(model_output_path, output_name)
    trainer.save_checkpoint(model_path)

    # Test the model
    trainer.test(model, test_loader)

    _, train_exp_stats = get_stats(model_path, train_loader, len(label_encoder.classes_), device)
    train_performance, _, _ = compute_experiment_performance(train_exp_stats)
    print(f"Train Performance: {train_performance}")

    _, val_exp_stats = get_stats(model_path, val_loader, len(label_encoder.classes_), device)
    val_performance, _, _ = compute_experiment_performance(val_exp_stats)
    print(f"Validation Performance: {val_performance}")

    _, test_exp_stats = get_stats(model_path, test_loader, len(label_encoder.classes_), device)
    test_performance, _, _ = compute_experiment_performance(test_exp_stats)
    print(f"Test Performance: {test_performance}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\miche\OneDrive\Desktop\tesi\repo-tesi\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Fold 1/12 -> Train: 60854 (exp: 61),  Val: 6751 (exp: 6),  Test: 5966 (exp: 6)
Epoch 24: 100%|██████████| 1902/1902 [01:12<00:00, 26.42it/s, v_num=0, train_loss=0.182, train_acc=1.000, val_loss=0.278, val_acc=0.865]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 1902/1902 [01:12<00:00, 26.40it/s, v_num=0, train_loss=0.182, train_acc=1.000, val_loss=0.278, val_acc=0.865]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

100%|██████████| 1926/1926 [01:26<00:00, 22.37it/s]


Train Performance: 0.9508196721311475


100%|██████████| 205/205 [01:00<00:00,  3.37it/s]


Validation Performance: 1.0


  0%|          | 0/170 [00:00<?, ?it/s]

In [7]:
#breakpoint
breakpoint_fold = 8

splits_path = r"C:\Users\miche\OneDrive\Desktop\tesi\repo-tesi\time_windows_approach\models\binaryETHvsACE\fold_splits_exp_names.json"
splits = json.load(open(splits_path, "r"))
splits = {int(k): v for k, v in splits.items()}

for fold in range(breakpoint_fold, folds+1):

    # Fold Splitting
    df_train = df[df["exp_name"].isin(splits[fold]["train"])]
    df_val = df[df["exp_name"].isin(splits[fold]["val"])]
    df_test = df[df["exp_name"].isin(splits[fold]["test"])]

    feature_columns = [col for col in df_train.columns if col not in ["label", "exp_name"]]
    scaler = StandardScaler()
    scaler.fit(df_train[feature_columns])
    mean = scaler.mean_
    std = scaler.scale_

    train_dataset = SensorDataset(df = df_train,
                                  label_encoder=label_encoder,
                                  exp_name_to_label=exp_name_to_label,
                                  mean=mean,
                                  std=std)
    
    val_dataset = SensorDataset(df = df_val,
                                label_encoder=label_encoder,
                                exp_name_to_label=exp_name_to_label,
                                mean=mean,
                                std=std)
    
    test_dataset = SensorDataset(df = df_test,
                                 label_encoder=label_encoder,
                                 exp_name_to_label=exp_name_to_label,
                                 mean=mean,
                                 std=std)
    
    print(f"\nFold {fold}/{folds} -> Train: {len(train_dataset)} (exp: {len(df_train['exp_name'].unique())}), ",
        f"Val: {len(val_dataset)} (exp: {len(df_val['exp_name'].unique())}), ",
        f"Test: {len(test_dataset)} (exp: {len(df_test['exp_name'].unique())})")
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, persistent_workers=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, persistent_workers=True)

    model = MLPClassifier(
        num_features=len(feature_columns),
        hidden_dim=hidden_dim,
        num_classes=2,
        lr=1e-4,
        dropout=0.2,
        weight_decay=1e-5
    )

    early_stop_callback = EarlyStopping(
        monitor="val_loss",
        patience=5,
        verbose=False,
        mode="min"
    )

    trainer = Trainer(
        max_epochs=MAX_EPOCHS,
        callbacks=[early_stop_callback],
        enable_model_summary=False,  # <- Hides the model summary
        accelerator="gpu" if torch.cuda.is_available() else "cpu",
        devices=1
    )

    trainer.fit(model, train_loader, val_loader)
    # Save the model
    output_name = f"binaryETHvsACE/binaryETHvsACE_MLP_h{hidden_dim}_{fold}.ckpt"
    model_path = os.path.join(model_output_path, output_name)
    trainer.save_checkpoint(model_path)

    # Test the model
    trainer.test(model, test_loader)

    _, train_exp_stats = get_stats(model_path, train_loader, len(label_encoder.classes_), device)
    train_performance, _, _ = compute_experiment_performance(train_exp_stats)
    print(f"Train Performance: {train_performance}")

    _, val_exp_stats = get_stats(model_path, val_loader, len(label_encoder.classes_), device)
    val_performance, _, _ = compute_experiment_performance(val_exp_stats)
    print(f"Validation Performance: {val_performance}")

    _, test_exp_stats = get_stats(model_path, test_loader, len(label_encoder.classes_), device)
    test_performance, _, _ = compute_experiment_performance(test_exp_stats)
    print(f"Test Performance: {test_performance}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\miche\OneDrive\Desktop\tesi\repo-tesi\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



Fold 8/12 -> Train: 61609 (exp: 61),  Val: 6544 (exp: 6),  Test: 5418 (exp: 6)
Epoch 24: 100%|██████████| 1926/1926 [00:47<00:00, 40.97it/s, v_num=7, train_loss=0.301, train_acc=0.778, val_loss=0.245, val_acc=0.886] 

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 1926/1926 [00:47<00:00, 40.96it/s, v_num=7, train_loss=0.301, train_acc=0.778, val_loss=0.245, val_acc=0.886]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Train Performance: 0.8787878787878788


100%|██████████| 220/220 [00:52<00:00,  4.21it/s]


Validation Performance: 0.6666666666666666


100%|██████████| 31/31 [00:55<00:00,  1.81s/it]

Test Performance: 0.0
